In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import os, re
import nltk
from bs4 import BeautifulSoup
sns.set()
sns.set_context('talk')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [8.0, 6.0]
colours = itertools.cycle(sns.color_palette('dark'))
palettes = itertools.cycle([sns.color_palette('PuBuGn_d'), sns.color_palette('Oranges_d'), sns.color_palette('GnBu_d'), sns.color_palette('Reds_d'), sns.color_palette('Blues_d'), sns.color_palette('PuRd_d')])
import csv
import spacy
spacy.load('en_core_web_sm')
nlp = spacy.load('en')
from collections import defaultdict

In [2]:
path = './Cornell_reviews/raw/'
files = [ path+d+'/'+f for d in os.listdir(path) if os.path.isdir(path+d) for f in os.listdir(path+d) ]
core_texts = []

def deHTML(comment):
    bs = BeautifulSoup(comment, 'lxml').text
    newlinetabs_removed = re.sub('[\n\t\r]', ' ', bs)
    extraws_removed = re.sub('\s\s+', ' ', newlinetabs_removed)
    res = re.sub('�', '\'', extraws_removed)
    res = re.sub('', '', res)
    return res.strip()

for f in files:
    with open(f, 'r') as fo:
        fo.readline()
        text = fo.read()
        core_texts.append(deHTML(text))

In [3]:
from collections import defaultdict

In [4]:
core_sets = []
for text in core_texts:
    core_sets.append(set(text.split()))

In [5]:
sim_dict = defaultdict(int)
similarities = []
for index,set1 in enumerate(core_sets):
    if index % 50 == 0: print(index)
    for index1,set2 in enumerate(core_sets):
        if (index != index1) and (sim_dict[(index,index1)] == 0):
            sim_score = len(set1.intersection(set2)) / len(set1.union(set2))
            similarities.append([core_texts[index], core_texts[index1], sim_score])
            
            sim_dict[(index,index1)] = 1
            sim_dict[(index1,index)] = 1

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950


In [6]:
sim_df = pd.DataFrame(similarities)

In [7]:
sim_df[(sim_df[2] > 0.5) & (sim_df[2] < 1)]

,0,1,2
53994,"first , let me assure you that no prior experi...","first , let me assure you that no prior experi...",0.997881
277935,"i still think that "" the bridge on the river k...",i still think that it doesn't deserve being th...,0.929078
509118,""" fun "" is a noun , and therefore cannot be co...",""" fun "" is a noun , and therefore cannot be co...",0.996546
1050339,event horizon is not your run-of-the-mill sci-...,event = horizon is not your run-of-the-mill sc...,0.982993
1652451,"ben kingsley ? "" my response would be that any...",his old associates want him for one last job a...,0.942500
1690618,"what better for the farrelly brothers , famous...","what better for the farrelly brothers , famous...",0.998296
1869343,they seem like dangerous rocks because they ki...,same old attitude . don't forget to recommend ...,0.929134
1978604,so i didn't expect too much from this . but i ...,so i didn't expect too much from this . but i ...,0.997403


In [8]:
for delete in list(sim_df[(sim_df[2] > 0.5) & (sim_df[2] < 1)][1].values):
    try:
        del core_texts[core_texts.index(delete)]
    except ValueError:
        pass

In [9]:
len(pd.DataFrame(list(set(core_texts))).index)
# should be 1992

1992

In [10]:
core_texts = list(set(core_texts))

In [11]:
with open('cornell_all_reviews.txt', 'w') as fo:
    for text in core_texts:
        fo.writelines(text+'\n\n')

In [ ]:
flat_list = [i[0] for i in pd.DataFrame(core_texts).values]
# genre_list = [i[0] for i in pd.DataFrame(core_texts).values]
        
tokens = []
lemma = []
pos = []
freq = 0
adj_count = 0

for index,doc in enumerate(nlp.pipe(flat_list, batch_size=50, n_threads=80)):
    print('still going:', index)
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
    else:
        # to make sure the indices will line up
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

In [ ]:
adv_adj_dict = defaultdict(int)
wordcount_dict = defaultdict(int)
adv_adj_list = []

for comment_index, comment in enumerate(pos):
    length = len(comment)
    genre = 'blah'
    if genre is not None:
        for word_index, part in enumerate(comment):
            
            if (part == 'ADJ'):
                adj_count += 1

            # for each adverb ending in -ly
            if (part == 'ADV') and tokens[comment_index][word_index][-2:] == 'ly':
                # if the next word is adj
                if (word_index+1 < length):
                    if (comment[word_index+1] == 'ADJ'):
                        word1 = tokens[comment_index][word_index].lower()
                        word2 = tokens[comment_index][word_index+1].lower()
                        
                        if word1 != 'only':                        
                            if (word1.isalpha() and word2.isalpha()):
                                # count it
                                freq = freq + 1
                                # dict key is 'word1 word2' in lowercase
                                key = tokens[comment_index][word_index].lower() + " " + tokens[comment_index][word_index+1].lower()
                                # add to the dict
                                adv_adj_dict[key] += 1
                            else:
                                print(word1, word2)

In [ ]:
pos_list = []
for comment_index,text in enumerate(pos):
    for word_index,part in enumerate(text):
        pos_list.append([comment_index, tokens[comment_index][word_index], part])

In [ ]:
pd.DataFrame(pos_list).rename(columns={0 : 'text_num', 1 : 'token', 2 : 'pos'}).to_csv('Cornell_pos.csv')

In [ ]:
adv_adj_dict

In [ ]:
print("Freq: " +str(freq))
print("Adjective count: " +str(adj_count))

print('Number of tokens')
print(sum([len(sublist) for sublist in pos]))

types = set([i for sublist in tokens for i in sublist])
print('Overall lexical diversity')
print(len(types) / len(tokens))

In [ ]:
# with open('adj_genre_dict.csv', 'w+') as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in adj_genre_dict.items():
#        writer.writerow([key, value])

# with open('advadj_genre_dict.csv', 'w+') as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in adv_adj_genre_dict.items():
#        writer.writerow([key, value])

with open('advadj_dict.csv', 'w+') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in adv_adj_dict.items():
       writer.writerow([key, value])
    
# with open('genre_wordcounts.csv', 'w+') as f:
#     writer = csv.writer(f)
#     for key, value in wordcount_dict.items():
#        writer.writerow([key, value])

In [ ]:
with open("overview.txt", "w+") as f:
    f.write("Frequency: " + str(freq) + "\n")
    f.write("Adjective count: " + str(adj_count) + "\n")
    f.write("Number of tokens: " + str(sum([len(sublist) for sublist in pos])) + "\n")
    types = set([i for sublist in tokens for i in sublist])
    f.write("Overall lexical diversity: " + str(len(types) / len(tokens)) + "\n")

In [ ]:
sum([len(sublist) for sublist in pos])